# IMPORTS

In [1]:
import random
import pickle
import glob

import random
import ast

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

from time import time
from time import sleep
from time import strftime
from time import localtime
from tqdm import tqdm

from datetime import date, timedelta, datetime

# from mydecorators import logger, stopwatch





In [2]:
def streak_counter(llist):
  result_dict = {
    ### 0 for Loss
    0: [],
    ### 1 for Win 
    1: []    
  }

  streak_size = 1

  for i in range(len(llist)):
    if i != len(llist)-1:
      if llist[i] != llist[i+1]:
        result_dict[llist[i]].append(streak_size)
        streak_size = 1

      elif llist[i] == llist[i+1]:
        streak_size += 1
    else:
      result_dict[llist[i]].append(streak_size)

  return result_dict


In [3]:
### function for calulating the average of number of consecutive results
def avg_cal(ddict):
  return np.mean(ddict[0]), np.mean(ddict[1])



In [4]:
### streak generator
#### using list.extend

def streak_gen(streak_size, result_size):    ### number of games
  result_streak = []

  n_games = result_size

  n_games_played = 0

  while n_games_played <= n_games:

    streak_size = streak_size
    streak = random.choices(['WS','LS'], weights=(0.5,0.5))[0]

    if streak == 'WS':
      wr = 0.85
      lr = 0.15
      # print(lr,wr)
      
      streak_games = random.choices([0,1], weights=(lr, wr), k=streak_size)
      result_streak.extend(streak_games)

    else:
      wr = 0.15
      lr = 0.85
      # print(lr,wr)
      streak_games = random.choices([0,1], weights=(lr, wr), k=streak_size)
      result_streak.extend(streak_games)

    n_games_played += streak_size


  return result_streak[:result_size]

# Generate GAME HISTORIES

## PerfectMM - One Million Games (to use as POPULATION)

In [5]:
start = time()
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))
print('\n')
##############################################################################################################################

### set random seed
random.seed(135)
np.random.seed(135)

# prob_list = np.linspace(0.49,0.51, 100)
prob_list = [0.5]

### when each game is between 49 ~ 51 % (independent)
#### 3000 games per player, 1000 players, totalling 3 million games played

avg_list_0 = []
avg_list_1 = []

one_million_games = []

# number_of_players = 1000
# games_per_player = 3000
number_of_players = 1
games_per_player = 1000000

for _ in range(number_of_players):
    result_n = []

    for _ in range(games_per_player):
        prob_of_0 = random.choice(prob_list)
        prob_of_1 = 1 - prob_of_0
        
        one_game = np.random.choice([0,1], 1, p = [prob_of_0, prob_of_1])[0]
        result_n.append(one_game)
    
    streak_counter_result = streak_counter(result_n)
    
    ### add to one_million_games (3000 players)
    one_million_games.append(result_n)
    # print(streak_counter_result)

    avg_cal_result = avg_cal(streak_counter_result)
    # print(avg_cal_result)
    avg_list_0.append(avg_cal_result[0])
    avg_list_1.append(avg_cal_result[1])
    
    
##############################################################################################################################
print('\n----------------------------------------------------')
print("done in %0.3fs." % (time() - start))
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))

2024-03-03 17:24:59



----------------------------------------------------
done in 7.146s.
2024-03-03 17:25:06


In [6]:
### check all records have 3000 games per player across 1000 players

### number of games per player
print(len(one_million_games[0]))
### number of players
print(len(one_million_games))

1000000
1


In [7]:
avg_cal_result

# (2.000536867578006, 2.0059375951537683)
# (1.9972764466627473, 2.0020676609036117)
# (1.9998959058989327, 2.003723365922794)
# (2.003923721884669, 2.0039557691305725)


(2.003923721884669, 2.0039557691305725)

In [8]:
print(np.mean(avg_list_0))
print(np.mean(avg_list_1))

# ### when n=1 games=1000000
# print(np.mean(streak_counter_result[0]))
# print(np.mean(streak_counter_result[1]))


# 2.0018657388686254
# 2.0018657388686254

2.003923721884669
2.0039557691305725


In [9]:
print(np.max(streak_counter(one_million_games[0][:])[0]))
print(np.min(streak_counter(one_million_games[0][:])[0]))

print(np.max(streak_counter(one_million_games[0][:])[1]))
print(np.min(streak_counter(one_million_games[0][:])[1]))



print(np.mean(streak_counter(one_million_games[0])[0]))
print(np.mean(streak_counter(one_million_games[0])[1]))

print(np.std(streak_counter(one_million_games[0])[0]))
print(np.std(streak_counter(one_million_games[0])[1]))

17
1
20
1
2.003923721884669
2.0039557691305725
1.4168229150676865
1.419380547217041


In [10]:
### Metrics from One million games to be used as Population

### mean of streak durations
## 0 for loss // 1 for win
pop_mean_0 = np.mean(streak_counter(one_million_games[0])[0])
pop_mean_1 = np.mean(streak_counter(one_million_games[0])[1])

### std of streak durations
pop_std_0 = np.std(streak_counter(one_million_games[0])[0])
pop_std_1 = np.std(streak_counter(one_million_games[0])[1])


print(f'pop_mean_0: {pop_mean_0}')
print(f'pop_mean_1: {pop_mean_1}')
print(f'pop_std_0: {pop_std_0}')
print(f'pop_std_1: {pop_std_1}')




pop_mean_0: 2.003923721884669
pop_mean_1: 2.0039557691305725
pop_std_0: 1.4168229150676865
pop_std_1: 1.419380547217041


## PerfectMM - 1000 players // 300 games per player (to compare against Population)

In [11]:
start = time()
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))
print('\n')
##############################################################################################################################

### set random seed
random.seed(6565)
np.random.seed(6565)

prob_list = [0.5]

### when each game is between 49 ~ 51 % (independent)
#### 3000 games per player, 1000 players, totalling 3 million games played

avg_list_0 = []
avg_list_1 = []

perfect_all_records = []

number_of_players = 1000
games_per_player = 300


for _ in range(number_of_players):
    result_n = []

    for _ in range(games_per_player):
        prob_of_0 = random.choice(prob_list)
        prob_of_1 = 1 - prob_of_0

        one_game = np.random.choice([0,1], 1, p = [prob_of_0, prob_of_1])[0]
        result_n.append(one_game)
    
    streak_counter_result = streak_counter(result_n)
    
    ### add to perfect_all_records (3000 players)
    perfect_all_records.append(result_n)
    # print(streak_counter_result)

    avg_cal_result = avg_cal(streak_counter_result)
    # print(avg_cal_result)
    avg_list_0.append(avg_cal_result[0])
    avg_list_1.append(avg_cal_result[1])
    
    
##############################################################################################################################
print('\n----------------------------------------------------')
print("done in %0.3fs." % (time() - start))
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))

2024-03-03 17:25:31



----------------------------------------------------
done in 2.168s.
2024-03-03 17:25:33


In [12]:
one_perfect = perfect_all_records[400]
len(one_perfect)


300

## NearPerfectMM - 1000 players // 300 games per player (to compare against Population)

In [13]:
start = time()
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))
print('\n')
##############################################################################################################################

### set random seed
random.seed(2237)
np.random.seed(2237)

prob_list = np.linspace(0.49,0.51, 100)

### when each game is between 49 ~ 51 % (independent)
#### 3000 games per player, 1000 players, totalling 3 million games played

avg_list_0 = []
avg_list_1 = []

nearperfect_all_records = []

number_of_players = 1000
games_per_player = 300


for _ in range(number_of_players):
    result_n = []

    for _ in range(games_per_player):
        prob_of_0 = random.choice(prob_list)
        prob_of_1 = 1 - prob_of_0

        one_game = np.random.choice([0,1], 1, p = [prob_of_0, prob_of_1])[0]
        result_n.append(one_game)
    
    streak_counter_result = streak_counter(result_n)
    
    ### add to nearperfect_all_records (3000 players)
    nearperfect_all_records.append(result_n)
    # print(streak_counter_result)

    avg_cal_result = avg_cal(streak_counter_result)
    # print(avg_cal_result)
    avg_list_0.append(avg_cal_result[0])
    avg_list_1.append(avg_cal_result[1])
    
    
##############################################################################################################################
print('\n----------------------------------------------------')
print("done in %0.3fs." % (time() - start))
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))

2024-03-03 17:25:33



----------------------------------------------------
done in 2.296s.
2024-03-03 17:25:35


In [14]:
one_nearperfect = nearperfect_all_records[400]
len(one_nearperfect)

300

## StreakMM - 1000 players // 300 games per player (to compare against Population)

In [15]:
start = time()
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))
print('\n')
##############################################################################################################################

### set random seed
random.seed(3331)
np.random.seed(3331)

### when streaks exist
### ws:ls = 50:50
### 85% of win/loss depending on the streak

avg_list_0 = []
avg_list_1 = []

streak_all_records = []
for _ in range(1000):
    result_n = streak_gen(6, 300)     # 3000 games played per player
    
    # streak_all_records.append(pd.Series(result_n))
    streak_all_records.append(result_n)
    
    streak_counter_result = streak_counter(result_n)
    # print(streak_counter_result)

    avg_cal_result = avg_cal(streak_counter_result)
    # print(avg_cal_result)
    avg_list_0.append(avg_cal_result[0])
    avg_list_1.append(avg_cal_result[1])
    
    
##############################################################################################################################
print('\n----------------------------------------------------')
print("done in %0.3fs." % (time() - start))
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))

2024-03-03 17:25:38



----------------------------------------------------
done in 0.111s.
2024-03-03 17:25:38


In [16]:
one_streakmm = streak_all_records[400]
len(one_streakmm)

300

# Analysis - Critical Count(s)

## The definition of CRITICAL COUNT

Critical Count : The frequency in which a record(eg. 300 games) yields streaks of either wins or defeats that surpasses the metrics provided by the Population data within a given sliding window(eg. 100 games for this notebook file). 

## PerfectMM vs Population

### Single PerfectMM vs Population

In [30]:
llen = len(one_perfect)

critical_count = 0

window_size = 100

for i in range(llen-window_size):
    # print(i , i+100)
    one_window = one_perfect[i:i+window_size]
    
    streak_counter_result = streak_counter(one_window)
    lose_streak = streak_counter_result[0]
    win_streak = streak_counter_result[1]
    
    ### ls = lose streak // ws = win streak
    ### means within one window (50 games)
    ls_mean = np.mean(lose_streak)
    ws_mean = np.mean(win_streak)
    
    ### standard deviations within one window (50 games)
    ls_std = np.std(lose_streak)
    ws_std = np.std(win_streak)
    
    
    ### z scores
    ### (x_bar_1 - mu1) / (std1/np.sqrt(n1))
    z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
    z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))
    
    
    if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
        critical_count += 1
        print(f'i ~ i+{window_size} : {i} ~ {i+window_size}, z_0 : {z_0}, z_1 : {z_1}')
        
    # print(f'z_1 : {z_1}')
    
print(f'critical_count : {critical_count}')

i ~ i+100 : 79 ~ 179, z_0 : 2.2441808531994556, z_1 : -0.4636926666562605
i ~ i+100 : 80 ~ 180, z_0 : 2.2441808531994556, z_1 : -0.4636926666562605
i ~ i+100 : 81 ~ 181, z_0 : 2.093702826560943, z_1 : -0.31348579171096774
i ~ i+100 : 83 ~ 183, z_0 : 2.1435774919100257, z_1 : -0.013072041820383698
i ~ i+100 : 86 ~ 186, z_0 : 1.9895583213310641, z_1 : 0.13713483312490835
i ~ i+100 : 87 ~ 187, z_0 : 1.9895583213310641, z_1 : 0.13713483312490835
i ~ i+100 : 92 ~ 192, z_0 : 2.1435774919100257, z_1 : -0.013072041820383698
i ~ i+100 : 94 ~ 194, z_0 : 2.093702826560943, z_1 : -0.012771494937112288
i ~ i+100 : 95 ~ 195, z_0 : 2.093702826560943, z_1 : -0.012771494937112288
i ~ i+100 : 96 ~ 196, z_0 : 2.093702826560943, z_1 : -0.012771494937112288
i ~ i+100 : 97 ~ 197, z_0 : 2.2975966624889885, z_1 : -0.16327891676567574
i ~ i+100 : 98 ~ 198, z_0 : 2.2975966624889885, z_1 : -0.16327891676567574
i ~ i+100 : 99 ~ 199, z_0 : 2.093702826560943, z_1 : -0.012771494937112288
critical_count : 13


### Critical Counts for all PerfectMM Records

In [20]:
crit_count_list = []

for item in perfect_all_records:
    one_record = item

    llen = len(one_record)

    critical_count = 0

    for i in range(llen-window_size):
        # print(i , i+100)
        one_window = one_record[i:i+window_size]

        streak_counter_result = streak_counter(one_window)
        lose_streak = streak_counter_result[0]
        win_streak = streak_counter_result[1]

        ### ls = lose streak // ws = win streak
        ### means within one window (50 games)
        ls_mean = np.mean(lose_streak)
        ws_mean = np.mean(win_streak)

        ### standard deviations within one window (50 games)
        ls_std = np.std(lose_streak)
        ws_std = np.std(win_streak)


        ### z scores
        ### (x_bar_1 - mu1) / (std1/np.sqrt(n1))
        z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
        z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))


        if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
            critical_count += 1
            # print(f'i ~ i+50 : {i} ~ {i+50}, z_0 : {z_0}, z_1 : {z_1}')

        # print(f'z_1 : {z_1}')

    crit_count_list.append(critical_count)
    # print(f'critical_count : {critical_count}')
    
# print(crit_count_list)

In [21]:
print(np.mean(crit_count_list))
print(np.std(crit_count_list))

18.638
25.401711674609647


In [22]:
perfectmm_pop_crit_count_mean = np.mean(crit_count_list)
perfectmm_pop_crit_count_std = np.std(crit_count_list)

print(perfectmm_pop_crit_count_mean)
print(perfectmm_pop_crit_count_std)

18.638
25.401711674609647


In [23]:
z_crit_count = (np.mean(crit_count_list) - perfectmm_pop_crit_count_mean) / (perfectmm_pop_crit_count_std/np.sqrt(len(crit_count_list)))
print(z_crit_count)

0.0


## NearPerfctMM vs Population

### Single NearPerfectMM vs Population

In [98]:
one_nearperfect = nearperfect_all_records[141]
len(one_nearperfect)

300

In [31]:
llen = len(one_nearperfect)

critical_count = 0

for i in range(llen-window_size):
    # print(i , i+100)
    one_window = one_nearperfect[i:i+window_size]
    
    streak_counter_result = streak_counter(one_window)
    lose_streak = streak_counter_result[0]
    win_streak = streak_counter_result[1]
    
    ### ls = lose streak // ws = win streak
    ### means within one window (50 games)
    ls_mean = np.mean(lose_streak)
    ws_mean = np.mean(win_streak)
    
    ### standard deviations within one window (50 games)
    ls_std = np.std(lose_streak)
    ws_std = np.std(win_streak)
    
    
    ### z scores
    ### (x_bar_1 - mu1) / (std1/np.sqrt(n1))
    z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
    z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))
    
    
    if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
        critical_count += 1
        print(f'i ~ i+{window_size} : {i} ~ {i+window_size}, z_0 : {z_0}, z_1 : {z_1}')
        
    # print(f'z_1 : {z_1}')
    
print(f'critical_count : {critical_count}')

i ~ i+100 : 92 ~ 192, z_0 : 2.0393098687946094, z_1 : 1.4425298493525325
i ~ i+100 : 93 ~ 193, z_0 : 2.254845777150218, z_1 : 1.247842693148089
i ~ i+100 : 94 ~ 194, z_0 : 2.0393098687946094, z_1 : 1.4425298493525325
i ~ i+100 : 95 ~ 195, z_0 : 2.0393098687946094, z_1 : 1.4425298493525325
i ~ i+100 : 96 ~ 196, z_0 : 2.254845777150218, z_1 : 1.247842693148089
i ~ i+100 : 97 ~ 197, z_0 : 2.254845777150218, z_1 : 1.604160734262413
i ~ i+100 : 98 ~ 198, z_0 : 2.0929231180579895, z_1 : 1.7657916191722947
i ~ i+100 : 101 ~ 201, z_0 : 2.0929231180579895, z_1 : 1.7657916191722947
i ~ i+100 : 102 ~ 202, z_0 : 2.0929231180579895, z_1 : 1.4053809926466996
i ~ i+100 : 108 ~ 208, z_0 : 1.7690777998735336, z_1 : 2.0890533889920566
i ~ i+100 : 109 ~ 209, z_0 : 1.7690777998735336, z_1 : 2.0890533889920566
i ~ i+100 : 112 ~ 212, z_0 : 1.7690777998735336, z_1 : 2.0890533889920566
i ~ i+100 : 113 ~ 213, z_0 : 1.4080191256199872, z_1 : 2.0890533889920566
i ~ i+100 : 114 ~ 214, z_0 : 1.4080191256199872, z_

### Critical Counts for all NearPerfectMM Records

In [25]:
crit_count_list = []

for item in nearperfect_all_records:
    one_record = item

    llen = len(one_record)

    critical_count = 0

    for i in range(llen-window_size):
        # print(i , i+100)
        one_window = one_record[i:i+window_size]

        streak_counter_result = streak_counter(one_window)
        lose_streak = streak_counter_result[0]
        win_streak = streak_counter_result[1]

        ### ls = lose streak // ws = win streak
        ### means within one window (50 games)
        ls_mean = np.mean(lose_streak)
        ws_mean = np.mean(win_streak)

        ### standard deviations within one window (50 games)
        ls_std = np.std(lose_streak)
        ws_std = np.std(win_streak)


        ### z scores
        ### (x_bar_1 - mu1) / (std1/np.sqrt(n1))
        z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
        z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))


        if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
            critical_count += 1
            # print(f'i ~ i+50 : {i} ~ {i+50}, z_0 : {z_0}, z_1 : {z_1}')

        # print(f'z_1 : {z_1}')

    crit_count_list.append(critical_count)
    # print(f'critical_count : {critical_count}')
    
# print(crit_count_list)

In [26]:
print(np.mean(crit_count_list))
print(np.std(crit_count_list))

18.083
25.345849975883624


In [27]:
z_crit_count = (np.mean(crit_count_list) - perfectmm_pop_crit_count_mean) / (perfectmm_pop_crit_count_std/np.sqrt(len(crit_count_list)))
print(z_crit_count)

-0.690923558174129


## StreakMM vs Population

### Single NearPerfectMM vs Population

In [103]:
one_streakmm = streak_all_records[666]
len(one_streakmm)

300

In [32]:
llen = len(one_streakmm)

critical_count = 0

for i in range(llen-window_size):
    # print(i , i+100)
    one_window = one_streakmm[i:i+100]
    
    streak_counter_result = streak_counter(one_window)
    lose_streak = streak_counter_result[0]
    win_streak = streak_counter_result[1]
    
    ### ls = lose streak // ws = win streak
    ### means within one window (50 games)
    ls_mean = np.mean(lose_streak)
    ws_mean = np.mean(win_streak)
    
    ### standard deviations within one window (50 games)
    ls_std = np.std(lose_streak)
    ws_std = np.std(win_streak)
    
    
    ### z scores
    ### (x_bar_1 - mu1) / (std1/np.sqrt(number_of_streak_durations))
    z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
    z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))
    
    
    if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
        critical_count += 1
        print(f'i ~ i+{window_size} : {i} ~ {i+window_size}, z_0 : {z_0}, z_1 : {z_1}')
        
    # print(f'z_1 : {z_1}')
    
print(f'critical_count : {critical_count}')

i ~ i+100 : 0 ~ 100, z_0 : 4.147243459493557, z_1 : 0.4863557384084469
i ~ i+100 : 1 ~ 101, z_0 : 4.313603177344575, z_1 : 0.3202957900333244
i ~ i+100 : 2 ~ 102, z_0 : 4.479962895195591, z_1 : 0.15423584165820056
i ~ i+100 : 3 ~ 103, z_0 : 4.479962895195591, z_1 : -0.1737789997462799
i ~ i+100 : 4 ~ 104, z_0 : 4.479962895195591, z_1 : 0.15423584165820056
i ~ i+100 : 5 ~ 105, z_0 : 4.479962895195591, z_1 : 0.15423584165820056
i ~ i+100 : 6 ~ 106, z_0 : 4.646322613046608, z_1 : 0.33025758809774536
i ~ i+100 : 7 ~ 107, z_0 : 4.479962895195591, z_1 : 0.15423584165820056
i ~ i+100 : 8 ~ 108, z_0 : 4.035995027164726, z_1 : 0.15423584165820056
i ~ i+100 : 9 ~ 109, z_0 : 3.874072368072498, z_1 : -0.012148114836398853
i ~ i+100 : 10 ~ 110, z_0 : 3.7121497089802706, z_1 : 0.14948277007348287
i ~ i+100 : 11 ~ 111, z_0 : 3.550227049888042, z_1 : 0.3111136549833632
i ~ i+100 : 12 ~ 112, z_0 : 3.814524023791523, z_1 : 0.47274453989324494
i ~ i+100 : 13 ~ 113, z_0 : 3.814524023791523, z_1 : 0.472744

### Critical Counts for all StreakMM Records

In [33]:
crit_count_list = []

for item in streak_all_records:
    one_record = item

    llen = len(one_record)

    critical_count = 0

    for i in range(llen-window_size):
        # print(i , i+100)
        one_window = one_record[i:i+window_size]

        streak_counter_result = streak_counter(one_window)
        lose_streak = streak_counter_result[0]
        win_streak = streak_counter_result[1]

        ### ls = lose streak // ws = win streak
        ### means within one window (50 games)
        ls_mean = np.mean(lose_streak)
        ws_mean = np.mean(win_streak)

        ### standard deviations within one window (50 games)
        ls_std = np.std(lose_streak)
        ws_std = np.std(win_streak)


        ### z scores
        ### (x_bar_1 - mu1) / (std1/np.sqrt(n1))
        z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
        z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))


        if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
            critical_count += 1
            # print(f'i ~ i+50 : {i} ~ {i+50}, z_0 : {z_0}, z_1 : {z_1}')

        # print(f'z_1 : {z_1}')

    crit_count_list.append(critical_count)
    # print(f'critical_count : {critical_count}')
    
# print(crit_count_list)

In [34]:
print(np.mean(crit_count_list))
print(np.std(crit_count_list))

194.937
12.846518244255913


In [35]:
z_crit_count = (np.mean(crit_count_list) - perfectmm_pop_crit_count_mean) / (perfectmm_pop_crit_count_std/np.sqrt(len(crit_count_list)))
print(z_crit_count)

219.47591420277487


## PerfectMM Critical Count Mean & Standard Deviation to be used as True Population  
-> Defined above in 'PerfectMM vs Population'  
-> Now renamed as ... true_pop_crit_count_mean & true_pop_crit_count_std  

In [108]:
# perfectmm_pop_crit_count_mean = np.mean(crit_count_list)
# perfectmm_pop_crit_count_std = np.std(crit_count_list)

print(perfectmm_pop_crit_count_mean)
print(perfectmm_pop_crit_count_std)

# perfectmm_pop_crit_count_mean = 22.763
# perfectmm_pop_crit_count_std = 20.68953433502069

# print(np.mean(crit_count_list))
# print(np.std(crit_count_list))

22.763
20.68953433502069


In [109]:
true_pop_crit_count_mean = perfectmm_pop_crit_count_mean
true_pop_crit_count_std = perfectmm_pop_crit_count_std

### Live Data - Bronze 0

In [113]:
parent_data_folder = 'live_data/raw'

datafiles_list = glob.glob(f'{parent_data_folder}/bronze/partition_0/*.txt')
print(datafiles_list[:5])


crit_count_list = []
for one_record_file in datafiles_list:
    with open(one_record_file, 'r') as datafile:
        one_record = datafile.read()
    
    one_record = ast.literal_eval(one_record)
    # print(f'length : {len(one_record)}')
    
    
    ### critical count analysis
    llen = len(one_record)

    critical_count = 0
    for i in range(llen-50):
        # print(i , i+50)
        one_window = one_record[i:i+50]
    
        streak_counter_result = streak_counter(one_window)
        lose_streak = streak_counter_result[0]
        win_streak = streak_counter_result[1]

        ### ls = lose streak // ws = win streak
        ### means within one window (50 games)
        ls_mean = np.mean(lose_streak)
        ws_mean = np.mean(win_streak)

        ### standard deviations within one window (50 games)
        ls_std = np.std(lose_streak)
        ws_std = np.std(win_streak)


        ### z scores
        ### (x_bar_1 - mu1) / (std1/np.sqrt(number_of_streak_durations))
        z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
        z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))

    

        if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
            critical_count += 1
            # print(f'i ~ i+50 : {i} ~ {i+50}, z_0 : {z_0}, z_1 : {z_1}')
        
    crit_count_list.append(critical_count)
    
    # print(f'critical_count : {critical_count}')
    # print('-----------------')
        # print(f'z_1 : {z_1}')
        
print(f'len before removal of 0s : {len(crit_count_list)}')
### remove 0s
crit_count_list = [x for x in crit_count_list if x != 0]
print(f'len after removal of 0s : {len(crit_count_list)}')
        
z_crit_count = (np.mean(crit_count_list) - true_pop_crit_count_mean) / (true_pop_crit_count_std/np.sqrt(len(crit_count_list)))
print(z_crit_count)
print('-------------------------')

['live_data/raw/bronze/partition_0\\03한주민.txt', 'live_data/raw/bronze/partition_0\\0서풋0.txt', 'live_data/raw/bronze/partition_0\\1161kk.txt', 'live_data/raw/bronze/partition_0\\12312415411213.txt', 'live_data/raw/bronze/partition_0\\123세봐넌디짐.txt']
len before removal of 0s : 813
len after removal of 0s : 625
8.766026197748092


### Live Data - Bronze 1

In [114]:
parent_data_folder = 'live_data/raw'

datafiles_list = glob.glob(f'{parent_data_folder}/bronze/partition_1/*.txt')
print(datafiles_list[:5])


crit_count_list = []
for one_record_file in datafiles_list:
    with open(one_record_file, 'r') as datafile:
        one_record = datafile.read()
    
    one_record = ast.literal_eval(one_record)
    # print(f'length : {len(one_record)}')
    
    
    ### critical count analysis
    llen = len(one_record)

    critical_count = 0
    for i in range(llen-50):
        # print(i , i+50)
        one_window = one_record[i:i+50]
    
        streak_counter_result = streak_counter(one_window)
        lose_streak = streak_counter_result[0]
        win_streak = streak_counter_result[1]

        ### ls = lose streak // ws = win streak
        ### means within one window (50 games)
        ls_mean = np.mean(lose_streak)
        ws_mean = np.mean(win_streak)

        ### standard deviations within one window (50 games)
        ls_std = np.std(lose_streak)
        ws_std = np.std(win_streak)


        ### z scores
        ### (x_bar_1 - mu1) / (std1/np.sqrt(number_of_streak_durations))
        z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
        z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))

    

        if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
            critical_count += 1
            # print(f'i ~ i+50 : {i} ~ {i+50}, z_0 : {z_0}, z_1 : {z_1}')
        
    crit_count_list.append(critical_count)
    
    # print(f'critical_count : {critical_count}')
    # print('-----------------')
        # print(f'z_1 : {z_1}')
        
print(f'len before removal of 0s : {len(crit_count_list)}')
### remove 0s
crit_count_list = [x for x in crit_count_list if x != 0]
print(f'len after removal of 0s : {len(crit_count_list)}')
        
z_crit_count = (np.mean(crit_count_list) - true_pop_crit_count_mean) / (true_pop_crit_count_std/np.sqrt(len(crit_count_list)))
print(z_crit_count)
print('-------------------------')

['live_data/raw/bronze/partition_1\\0l븥린.txt', 'live_data/raw/bronze/partition_1\\0tlqkf0.txt', 'live_data/raw/bronze/partition_1\\1017번째계정.txt', 'live_data/raw/bronze/partition_1\\1249559del.txt', 'live_data/raw/bronze/partition_1\\145cm.txt']
len before removal of 0s : 985
len after removal of 0s : 703
8.388446210463274
-------------------------


### For All the REST of ELOS & PARTITIONS

In [119]:
start = time()
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))
print('\n')
##############################################################################################################################



### https://stackoverflow.com/questions/74949556/poetry-fails-to-install-tensorflow

### Training image data path
# parent of ['saved_lists/perfectmm/', 'saved_lists/nearperfectmm/', 'saved_lists/streakmm/']
parent_data_folder = 'live_data/raw'
# print(parent_data_folder)

elo_list = glob.glob(f'{parent_data_folder}/*') 

# print(elo_list)

for item in elo_list:
    partitions_list = glob.glob(f'{item}/*')
    print(item)
    print(partitions_list)
    
    for partition in partitions_list:
        datafiles_list = glob.glob(f'{partition}/*.txt')
        print(datafiles_list[:2])
        
        crit_count_list = []
        for one_record_file in datafiles_list:
            with open(one_record_file, 'r') as datafile:
                one_record = datafile.read()
            
            one_record = ast.literal_eval(one_record)
            
            ### critical count analysis
            llen = len(one_record)
            
            critical_count = 0
            for i in range(llen-50):
                # print(i , i+50)
                one_window = one_record[i:i+50]
                
                streak_counter_result = streak_counter(one_window)
                lose_streak = streak_counter_result[0]
                win_streak = streak_counter_result[1]

                ### ls = lose streak // ws = win streak
                ### means within one window (50 games)
                ls_mean = np.mean(lose_streak)
                ws_mean = np.mean(win_streak)

                ### standard deviations within one window (50 games)
                ls_std = np.std(lose_streak)
                ws_std = np.std(win_streak)


                ### z scores
                ### (x_bar_1 - mu1) / (std1/np.sqrt(number_of_streak_durations))
                z_0 = (ls_mean - pop_mean_0) / (pop_std_0/np.sqrt(len(lose_streak)))
                z_1 = (ws_mean - pop_mean_1) / (pop_std_1/np.sqrt(len(win_streak)))
                
                if abs(z_0) >= 1.96 or abs(z_1) >= 1.96:
                    critical_count += 1
                    # print(f'i ~ i+50 : {i} ~ {i+50}, z_0 : {z_0}, z_1 : {z_1}')
                    
            crit_count_list.append(critical_count)

    
        print(f'len before removal of 0s : {len(crit_count_list)}')
        ### remove 0s
        crit_count_list = [x for x in crit_count_list if x != 0]
        print(f'len after removal of 0s : {len(crit_count_list)}')

        z_crit_count = (np.mean(crit_count_list) - true_pop_crit_count_mean) / (true_pop_crit_count_std/np.sqrt(len(crit_count_list)))
        print(z_crit_count)
        print('--------End of One Partition-----------------')
    
    
    
    
    
    
    
##############################################################################################################################
print('\n----------------------------------------------------')
print("done in %0.3fs." % (time() - start))
print(strftime('%Y-%m-%d %H:%M:%S', localtime()))

2023-12-17 20:40:14


live_data/raw\bronze
['live_data/raw\\bronze\\partition_0', 'live_data/raw\\bronze\\partition_1', 'live_data/raw\\bronze\\partition_2', 'live_data/raw\\bronze\\partition_3', 'live_data/raw\\bronze\\partition_4']
['live_data/raw\\bronze\\partition_0\\03한주민.txt', 'live_data/raw\\bronze\\partition_0\\0서풋0.txt']
len before removal of 0s : 813
len after removal of 0s : 625
8.766026197748092
--------End of One Partition-----------------
['live_data/raw\\bronze\\partition_1\\0l븥린.txt', 'live_data/raw\\bronze\\partition_1\\0tlqkf0.txt']
len before removal of 0s : 985
len after removal of 0s : 703
8.388446210463274
--------End of One Partition-----------------
['live_data/raw\\bronze\\partition_2\\00나리00.txt', 'live_data/raw\\bronze\\partition_2\\01031784315.txt']
len before removal of 0s : 992
len after removal of 0s : 879
11.453049635071576
--------End of One Partition-----------------
['live_data/raw\\bronze\\partition_3\\0120123.txt', 'live_data/raw\\bronze\\partition_